In [8]:
from itertools import product, combinations
import keyboard
import pyautogui as g
import time
import pytesseract

In [ ]:
pytesseract.pytesseract.tesseract_cmd = 'C:\Program Files\Tesseract-OCR/tesseract'

suits = ('o','e','c','p')
values = (2,3,4,5,6,7,8,9,10,11,12,13,14)
turno = 1
pessoas = 3
pote = 0
pagar = 0
jogar = 0
mao = ['00','00']
mesa = ['00','00','00','00','00']
minhasprobs = [0,0,0,0,0,0,0,0,0]
adversprobs = [0,0,0,0,0,0,0,0,0]
bb = 100
print('Funcionando!!!')

def Calcular():
    global turno
    global mao
    global mesa
    global minhasprobs
    global adversprobs
    global jogar
    
    # criar o array de cartas a partir das combinações entre numeros e naipes
    deck = [str(card[0]) + card[1] for card in list(product(values, suits))]
    
    # remover cartas que já estão na mão e/ou na mesa do deck
    deck.pop(deck.index(mao[0]))
    deck.pop(deck.index(mao[1]))
    
    # dependendo do turno, existem mais cartas ou menos cartas já jogadas na mesa para serem removidas
    if turno > 0:
        deck.pop(deck.index(mesa[0]))
        deck.pop(deck.index(mesa[1]))
        deck.pop(deck.index(mesa[2]))
        # numcarta é o número de cartas para serem testadas para completar as 7
        numcarta = 2
        if turno > 1:
            deck.pop(deck.index(mesa[3]))
            numcarta = 1
            if turno > 2:
                deck.pop(deck.index(mesa[4]))
                numcarta = 0
    
    # o loop roda 1 vez para calcular as minhas probabilidades e depois as dos adversários
    # para x == 0 são calculadas as minhas probabilidades
    for x in range(2):
        # variáveis que determinam quantas vezes essas jogadas foram obtidas nos testes
        total = 0
        flush = 0
        quad = 0
        full = 0
        trin = 0
        pare = 0
        parr = 0
        nada = 0
        sequencia = 0
        strf = 0
        
        if x == 1:
            # para calcular as probabilidades alheias, existem 2 cartas a
            # mais para serem testadas, já que não sabemos a mão deles
            numcarta = numcarta + 2
            print('adversarios probs:')
        else:
            print('minhas probs:')
        
        # todas as combinações de cartas serão testadas de acordo com o número de cartas que faltam
        for i in list(combinations(deck,numcarta)):
            par = 0
            pares = 0
            trinca = 0
            fullHouse = 0
            quadra = 0
            sequ = 0
            naipeFlush = '0'
            strflush = 0
            total = total + 1
            
            # as cartas usadas na jogada poderam conter cartas da mao, da mesa e as cartas testadas
            # dependendo do turno e de quem são as probabilidades
            if x == 0:
                if numcarta == 2:
                    jogada = [mesa[0],mesa[1],mesa[2], mao[0], mao[1], i[0], i[1]]
                elif numcarta == 1:
                    jogada = [mesa[0],mesa[1],mesa[2], mesa[3], mao[0], mao[1], i[0]]
                elif numcarta == 0:
                    jogada = [mesa[0],mesa[1],mesa[2], mesa[3], mesa[4], mao[0], mao[1]]
            else:  
                if numcarta == 4:
                    jogada = [mesa[0],mesa[1],mesa[2], i[0], i[1], i[2], i[3]]
                elif numcarta == 3:
                    jogada = [mesa[0],mesa[1],mesa[2],mesa[3], i[0], i[1], i[2]]
                elif numcarta == 2:
                    jogada = [mesa[0],mesa[1],mesa[2], mesa[3], mesa[4],i[0], i[1]]
            
            # as cartas testadas serão colocadas em ordem em um novo array de números inteiros
            # se os digitos não forem convertidos em inteiros, o sort() só detecta o primeiro digito para determinar a ordenação
            # isso serve para poder testar sequência ou straight flushs
            seq = [int(jogada[0][:-1]),int(jogada[1][:-1]),int(jogada[2][:-1]),int(jogada[3][:-1]),int(jogada[4][:-1]),int(jogada[5][:-1]),int(jogada[6][:-1])]
            seq.sort()

            # ele contara quantas cartas tem de cada naipe e depois determinará se houve um flush
            countFlush = [0,0,0,0]
            for j in jogada:
                if j[-1] == 'o':
                    countFlush[0] = countFlush[0] + 1
                elif j[-1] == 'e':
                    countFlush[1] = countFlush[1] + 1
                elif j[-1] == 'c':
                    countFlush[2] = countFlush[2] + 1
                elif j[-1] == 'p':
                    countFlush[3] = countFlush[3] + 1
            # armazenará de qual naipe é o flush para calcular depois se houve straight flush
            if countFlush[0] > 4:
                    naipeFlush = 'o'
            elif countFlush[1] > 4:
                naipeFlush = 'e'
            elif countFlush[2] > 4:
                naipeFlush = 'c'
            elif countFlush[3] > 4:
                naipeFlush = 'p'

            # ele verificará se há alguma sequencia, seja começando da 1,2 ou começando a partir da 3 carta
            # para isso, ele primeiro removera as cartas repetidas
            straight = list(set(seq))
            if len(straight) > 4:
                if len(straight) == 5:
                    if straight[0] + 4 == straight[4]:
                        sequ = 1
                elif len(straight) == 6:
                    if straight[0] + 4 == straight[4] or straight[1] + 4 == straight[5]:
                        sequ = 1
                elif len(straight) == 7:
                    if straight[0] + 4 == straight[4] or straight[1] + 4 == straight[5] or straight[2] + 4 == straight[6]:
                        sequ = 1

            # aqui são detectados pares, trincas e quadras
            nums = [0,0,0,0,0,0,0]
            # ele conta quantas copias de carta tem para cada carta
            for p in seq:
                # o array nums conterá as quantidades das cartas
                nums[seq.index(p)] = seq.count(p)
            if 4 in nums:
                quadra = 1
            elif 3 in nums:
                nums.pop(nums.index(3))
                if 2 in nums or 3 in nums:
                    fullHouse = 1
                else:
                    trinca = 1
            elif 2 in nums:
                nums.pop(nums.index(2))
                # o código removerá o 2 do array de quatidades
                # se mesmo assim ainda tiver outra dupla, o código detectará pares
                if 2 in nums:
                    pares = 1
                else:
                    par = 1

            
            seqflush = []
            # conferirá se o código anterior indicou sequencia e flush
            if naipeFlush != '0' and sequ == 1:
                # ele vai adicionar ao array cada carta do naipe do flush
                for i in jogada:
                    if naipeFlush == i[-1]:
                        seqflush.append(i)
                # se realmente tiver mais de 4 cartas do mesmo naipe, ele testará se houve sequência
                if len(seqflush) > 4:
                    seqflush2 = []
                    for k in seqflush:
                        # ele adicionará apenas os dígitos a um array de cartas do mesmo naipe
                        seqflush2.append(int(k[:-1]))
                    # vai ordenar essas cartas do mesmo naipe
                    seqflush2.sort()
                    # ele verificará se há alguma sequencia, seja começando da 1,2 ou começando a partir da 3 carta
                    if len(seqflush2) == 5:
                        if seqflush2[0] + 4 == seqflush2[4]:
                            strflush = 1
                    elif len(seqflush2) == 6:
                        if seqflush2[0] + 4 == seqflush2[4] or seqflush2[1] + 4 == seqflush2[5]:
                            strflush = 1
                    elif len(seqflush2) == 7:
                        if seqflush2[0] + 4 == seqflush2[4] or seqflush2[1] + 4 == seqflush2[5] or seqflush2[2] + 4 == seqflush2[6]:
                            strflush = 1


            # ele vai ver qual foi a melhor jogada que o teste encontrou
            if strflush == 1:
                strf = strf + 1

            elif quadra == 1:
                quad = quad + 1

            elif fullHouse == 1:
                full = full + 1

            elif naipeFlush != '0':
                flush = flush + 1

            elif sequ == 1:
                sequencia = sequencia + 1

            elif trinca == 1:
                trin = trin + 1

            elif pares == 1:
                pare = pare + 1

            elif par == 1:
                parr = parr + 1

            else:
                nada = nada + 1
        
        # a partir dos testes, serão lançadas as minhas probabilidades e as dos adversários
        if x == 0:
            print('straight flush :', round((strf/total)*100, 2), '%')
            minhasprobs[8] = round((strf/total)*100)
            print('quadra :', round((quad/total)*100, 2), '%')
            minhasprobs[7] = round((quad/total)*100)
            print('full house :', round((full/total)*100, 2), '%')
            minhasprobs[6] = round((full/total)*100)
            print('flush :', round((flush/total)*100, 2), '%')
            minhasprobs[5] = round((flush/total)*100)
            print('sequencia :', round((sequencia/total)*100, 2), '%')
            minhasprobs[4] = round((sequencia/total)*100)
            print('trinca :', round((trin/total)*100, 2), '%')
            minhasprobs[3] = round((trin/total)*100)
            print('pares :', round((pare/total)*100, 2), '%')
            minhasprobs[2] = round((pare/total)*100)
            print('par :', round((parr/total)*100, 2), '%')
            minhasprobs[1] = round((parr/total)*100)
            print('nada :', round((nada/total)*100, 2), '%')
            minhasprobs[0] = round((nada/total)*100)
        else:
            # essas são as probabilidades de pelo menos 1 adversário ter a jogada
            print('straight flush :', round((strf/total)*100, 2), '%')
            adversprobs[8] = round((strf/total)*100)
            print('quadra :', round((1 - ((1 - quad/total)**pessoas))*100, 2), '%')
            adversprobs[7] = round((quad/total)*100)
            print('full house :', round((1 - ((1 - full/total)**pessoas))*100, 2), '%')
            adversprobs[6] = round((full/total)*100)
            print('flush :', round((1 - ((1 - flush/total)**pessoas))*100, 2), '%')
            adversprobs[5] = round((flush/total)*100)
            print('sequencia :', round((1 - ((1 - sequencia/total)**pessoas))*100, 2), '%')
            adversprobs[4] = round((sequencia/total)*100)
            print('trinca :', round((1 - ((1 - trin/total)**pessoas))*100, 2), '%')
            adversprobs[3] = round((trin/total)*100)
            print('pares :', round((1 - ((1 - pare/total)**pessoas))*100, 2), '%')
            adversprobs[2] = round((pare/total)*100)
            print('par :', round((1 - ((1 - parr/total)**pessoas))*100, 2), '%')
            adversprobs[1] = round((parr/total)*100)
            print('nada :', round((1 - ((1 - nada/total)**pessoas))*100, 2), '%')
            adversprobs[0] = round((nada/total)*100)
            PosFlop()

# essa função detecta na tela o valor do pote e o valor que eu preciso pagar pra ir para o próximo round
def scrap():
    global pote
    global pagar
    global bb
    # captura a regiao onde fica o valor do pote
    img = g.screenshot(region=(890,370, 160, 35))
    pote = pytesseract.image_to_string(img)
    # o que foi capturado e não é número vai ser cortado da string
    for i in pote:
        if i.isdigit() == False:
            pote = pote.replace(i,"")
    pote = int(pote)
    print('pote :', pote)
    
    
    img = g.screenshot(region=(1380,890, 130, 25))
    aposta = pytesseract.image_to_string(img)
    if len(aposta) > 6:
        img = g.screenshot(region=(1230,900, 80, 30))
        passo = pytesseract.image_to_string(img)
        if len(passo) != 0:
            pagar = 0
        else:
            img = g.screenshot(region=(1220,913, 100, 35))
            pagar = pytesseract.image_to_string(img)
            # tentar tirar outras fotos para ver se assim identifica o valor
            if pagar == '':
                img = g.screenshot(region=(1225,910, 95, 30))
                pagar = pytesseract.image_to_string(img)
                print('bugou o pagar 1')
            if pagar == '':
                img = g.screenshot(region=(1225,913, 95, 40))
                pagar = pytesseract.image_to_string(img)
                print('bugou o pagar 2')
            if pagar == '':
                img = g.screenshot(region=(1220,914, 100, 28))
                pagar = pytesseract.image_to_string(img)
                print('bugou o pagar 3')
            if pagar == '':
                print('bugou o pagar')
    
    else:
        pagar = bb*100
    pagar = int(pagar)
    print('pagar :', pagar)
        
def preFlop():
    global bb
    global pagar
    global pote
    global mao
    apostar = 0
    
    # o que pagou o bigblind e o smallblind são contabilizados como se já tivessem jogado
    faltam = 2
    # se eu tiver pago o bigblind não falta ninguem
    if pagar == 0:
        faltam = 0
    # se eu tiver pago o smallblind falta o pagador do bigblind
    elif pagar == bb/2:
        faltam = 1
    else:
        if g.pixelMatchesColor(485, 530, (163, 69, 67), tolerance = 10):
            if g.pixelMatchesColor(665, 560, (7, 91, 32), tolerance = 10):
                faltam = faltam + 1
        if g.pixelMatchesColor(510, 270, (163, 69, 67), tolerance = 10):
            if g.pixelMatchesColor(680, 395, (7, 92, 32), tolerance = 10):
                faltam = faltam + 1
        if g.pixelMatchesColor(935, 195, (163, 69, 67), tolerance = 10):
            if g.pixelMatchesColor(1015, 310, (6, 86, 30), tolerance = 10):
                faltam = faltam + 1
        if g.pixelMatchesColor(1340, 270, (163, 69, 67), tolerance = 10):
            if g.pixelMatchesColor(1250, 370, (7, 91, 31), tolerance = 10):
                faltam = faltam + 1
        if g.pixelMatchesColor(1380, 530, (163, 69, 67), tolerance = 10):
            if g.pixelMatchesColor(1285, 560, (7, 93, 32), tolerance = 10):
                faltam = faltam + 1
        print('faltam :', faltam)
    
    rating = 0
    maos = [[14,14],[13,13],[12,12],[14,13,1],[11,11],
           [14,12,1],[13,12,1],[14,11,1],[13,11,1],[10,10],
           [14,13],[14,10,1],[12,11,1],[13,10,1],[12,10,1],
           [11,10,1],[9,9],[14,12],[14,9,1],[13,12],
           [8,8],[13,9,1],[10,9,1],[14,8,1],[13,9,1],
           [11,9,1],[14,11],[14,5,1],[7,7],[14,7,1],
           [13,11],[14,4,1],[14,3,1],[14,6,1],[12,11],
           [6,6],[12,8,1],[10,8,1],[14,2,1],[9,8,1],
           [11,8,1],[14,10],[12,8,1],[13,7,1],[13,10],
           [5,5],[11,10],[8,7,1],[12,10],[4,4],
           [3,3],[2,2],[13,6,1],[9,7,1],[13,5,1],
           [7,6,1],[10,7,1],[13,4,1],[13,3,1],[13,2,1],
           [12,7,1],[8,6,1],[6,5,1],[11,7,1],[5,4,1],
           [12,6,1],[7,5,1],[9,6,1],[12,5,1],[6,4,1]]
    for i in maos:
        if int(mao[0][:-1]) == i[0] and int(mao[1][:-1]) == i[1] or int(mao[1][:-1]) == i[0] and int(mao[0][:-1]) == i[1]:
            if len(i) == 3:
                if mao[0][-1] == mao[1][-1]:
                    rating = 169 - maos.index(i)
            else:
                rating = 169 - maos.index(i)
    print('rating :', rating)
    
    
    pago = 0
    aposto = 0
    if faltam > 3:
        if rating > 165:
            pago = bb*100
            aposto = 0
        elif rating > 156:
            pago = bb*2
            aposto = 0
        else:
            pago = 0
            aposto = 0
    elif faltam > 1:
        if rating > 165:
            pago = bb*100
            aposto = bb*4
        elif rating > 153:
            pago = bb*3
            aposto = bb*2
        elif rating > 116:
            pago = bb
            aposto = 0
        else:
            pago = 0
            aposto = 0
    else:
        if rating > 165:
            pago = bb*100
            aposto = bb*4
        elif rating > 147:
            pago = bb*5
            aposto = bb*2
        elif rating > 108:
            pago = bb
            aposto = 0
        else:
            pago = 0
            aposto = 0
    
    if aposto > pagar:
        g.moveTo(1220,845)
        g.doubleClick()
        g.write(str(aposto))
        g.moveTo(1460,910)
        g.click()
    elif pago >= pagar:
        if pagar == 100*bb:
            g.moveTo(1460,910)
            g.click()
        else:
            g.moveTo(1270,910)
            g.click()
    else:
        g.moveTo(1060,910)
        g.click()
    
    g.moveTo(1450,450)
    Loading()

def PosFlop():
    global mao
    global mesa
    global turno
    global minhasprobs
    global adversprobs
    global pote
    global pagar
    global bb
    
    
    faltam = 0
    if g.pixelMatchesColor(485, 530, (163, 69, 67), tolerance = 10):
        if g.pixelMatchesColor(665, 560, (7, 91, 32), tolerance = 10):
            faltam = faltam + 1
    if g.pixelMatchesColor(510, 270, (163, 69, 67), tolerance = 10):
        if g.pixelMatchesColor(680, 395, (7, 92, 32), tolerance = 10):
            faltam = faltam + 1
    if g.pixelMatchesColor(935, 195, (163, 69, 67), tolerance = 10):
        if g.pixelMatchesColor(1015, 310, (6, 86, 30), tolerance = 10):
            faltam = faltam + 1
    if g.pixelMatchesColor(1340, 270, (163, 69, 67), tolerance = 10):
        if g.pixelMatchesColor(1250, 370, (7, 91, 31), tolerance = 10):
            faltam = faltam + 1
    if g.pixelMatchesColor(1380, 530, (163, 69, 67), tolerance = 10):
        if g.pixelMatchesColor(1285, 560, (7, 93, 32), tolerance = 10):
            faltam = faltam + 1
    print('faltam :', faltam)
    
    winrate = 0
    drawrate = 0
    print(minhasprobs, adversprobs)
    for i in range(9):
        drawrate = drawrate + minhasprobs[i]*adversprobs[i]/100
        for j in range(i):
            winrate = winrate + minhasprobs[i]*adversprobs[j]/100
    print('winrate: ', winrate)
    print('drawrate: ', drawrate)
    
    # detectar as combinações que eu tenho na mão e na mesa
    tempar = 0
    if minhasprobs[0] == 0:
        # tenho algo
        if minhasprobs[1] > 0
            # esse algo é um par
            if adversprobs[0] == 0 and adversprobs[1] > 0:
                # todos tem par
                tempar = 0
            else:
                tempar = 1
            
    order = [int(mesa[0][:-1]), int(mesa[1][:-1]), int(mesa[2][:-1]), int(mesa[3][:-1]), int(mesa[4][:-1])]
    order.sort()
    order2 = list(set(order))
    
    if tempar == 1:
        if mao[0][:-1] == mao[1][:-1]:
            cartapar = int(mao[0][:-1])
            if cartapar > order2[0]
                numpar = 0
            elif cartapar > order2[1]
                numpar = 1
            elif cartapar > order2[2]
                numpar = 2
            else:
                numpar = 3
        else:
            numpar = 1
            for a in mao:
                for b in order2:
                    if int(a[:-1]) == b:
                        cartapar = b
                        break
                    else: numpar = numpar + 1
    
    # se o par for bom o winrate sobe
    winrate = winrate + drawrate*(2-numpar)/6
    
    #se for uns dos últimos a apostar, o winrate sobe
    winrate = winrate*(9/8+faltam)
    
    if turno == 1:
        if winrate > 85:
            pago = bb*100
            aposto = 0
        elif winrate > 80:
            pago = bb*25
            aposto = bb*1
        elif winrate > 73:
            pago = bb*15
            aposto = bb*4
        elif winrate > 66:
            pago = bb*12
            aposto = bb*3
        elif winrate > 59:
            pago = bb*9
            aposto = bb*2
        elif winrate > 52:
            pago = bb*7
            aposto = bb*2
        elif winrate > 45:
            pago = bb*5
            aposto = bb*1
        elif winrate > 38:
            pago = bb*3
            aposto = bb*1
        elif winrate > 31:
            pago = bb*1
            aposto = 0
        else:
            pago = 0
            aposto = 0
        
    elif turno == 2:
        if winrate > 87:
            pago = bb*100
            aposto = bb*30
        elif winrate > 80:
            pago = bb*100
            aposto = bb*22
        elif winrate > 73:
            pago = bb*35
            aposto = bb*13
        elif winrate > 66:
            pago = bb*28
            aposto = bb*8
        elif winrate > 59:
            pago = bb*18
            aposto = bb*6
        elif winrate > 52:
            pago = bb*8
            aposto = bb*3
        elif winrate > 45:
            pago = bb*4
            aposto = bb*1
        elif winrate > 38:
            pago = bb*2
            aposto = 0
        else:
            pago = 0
            aposto = 0
            
    elif turno == 3:
        if winrate > 87:
            pago = bb*100
            aposto = 0
        elif winrate > 80:
            pago = bb*25
            aposto = bb*1
        elif winrate > 73:
            pago = bb*15
            aposto = bb*4
        elif winrate > 66:
            pago = bb*12
            aposto = bb*3
        elif winrate > 59:
            pago = bb*9
            aposto = bb*2
        elif winrate > 52:
            pago = bb*7
            aposto = bb*2
        elif winrate > 45:
            pago = bb*5
            aposto = bb*1
        elif winrate > 38:
            pago = bb*3
            aposto = bb*1
        elif winrate > 31:
            pago = bb*1
            aposto = 0
        else:
            pago = 0
            aposto = 0
        
    # se começar com algo muito bom, como uma trinca ou fullhouse, não aposte nada para blefar.
    if turno == 1 and minhasprobs[2] == 0:
        aposto = 0
        
        
    if aposto > pagar:
        g.moveTo(1220,845)
        g.doubleClick()
        g.write(str(aposto))
        g.moveTo(1460,910)
        g.click()
    elif pago >= pagar:
        if pagar == 100*bb:
            g.moveTo(1460,910)
            g.click()
        else:
            g.moveTo(1270,910)
            g.click()
    else:
        g.moveTo(1060,910)
        g.click()

    g.moveTo(1450,450)
        
    Loading()
        
def Loading():
    global pessoas
    global turno
    global mao
    global mesa
    global jogar
    jogar = 0
    while jogar == 0:
        # quando um pixel vermelho escuro aparecer na tela, é por que o botão de apostas apareceu
        # isso significa que está na hora do robo tomar alguma decisão
        if g.pixelMatchesColor(1440, 915, (85, 14, 9), tolerance = 10):
            jogar = 1
            
            # ele detectará a cor de 1 pixel para ver se os adversários já desistiram da mão ou estão no jogo ainda
            # com isso, ele saberá contra quantos ele está jogando
            pessoas = 0
            if g.pixelMatchesColor(485, 530, (163, 69, 67), tolerance = 10):
                pessoas = pessoas + 1
            if g.pixelMatchesColor(510, 270, (163, 69, 67), tolerance = 10):
                pessoas = pessoas + 1
            if g.pixelMatchesColor(935, 195, (163, 69, 67), tolerance = 10):
                pessoas = pessoas + 1
            if g.pixelMatchesColor(1340, 270, (163, 69, 67), tolerance = 10):
                pessoas = pessoas + 1
            if g.pixelMatchesColor(1380, 530, (163, 69, 67), tolerance = 10):
                pessoas = pessoas + 1

            scrap()
            # ele detectara um pixel para saber se a mesa continua verde
            # se não, é porque colocaram 1 carta em cima daquele pixel, indicando que já passou o preflop
            if g.pixelMatchesColor(835, 465, (7, 101, 35), tolerance = 10):
                turno = 0
                # no preflop ele verá as cartas que possui
                # aqui ele testa todas os números para as 2 cartas para saber o valor de cada uma
                for a in range(13):
                    arquivo = 'PokerStarsImages/cartas/' + str(a + 2) + '.jpg'
                    if g.locateOnScreen(arquivo, confidence = 0.75, region = (897,635,30,40), grayscale = True):
                        mao[0] = str(a+2)

                for b in range(13):
                    arquivo = 'PokerStarsImages/cartas/' + str(b + 2) + '.jpg'
                    if g.locateOnScreen(arquivo, confidence = 0.75, region = (972,635,30,40), grayscale = True):
                        mao[1] = str(b+2)

                if g.locateOnScreen('PokerStarsImages/ouros2.png', confidence = 0.80, region = (900,670,30,30), grayscale = True):
                    mao[0] = mao[0] + 'o'  
                elif g.locateOnScreen('PokerStarsImages/copas2.png', confidence = 0.8, region = (900,670,30,30), grayscale = True):
                    mao[0] = mao[0] + 'c'
                elif g.locateOnScreen('PokerStarsImages/espadas2.png', confidence = 0.85, region = (900,670,30,30), grayscale = True):
                    mao[0] = mao[0] + 'e'
                elif g.locateOnScreen('PokerStarsImages/paus2.png', confidence = 0.8, region = (900,670,30,30), grayscale = True):
                    mao[0] = mao[0] + 'p' 

                if g.locateOnScreen('PokerStarsImages/ouros3.png', confidence = 0.85, region = (974,670,30,30), grayscale = True):
                    mao[1] = mao[1] + 'o'  
                elif g.locateOnScreen('PokerStarsImages/copas3.png', confidence = 0.8, region = (974,670,30,30), grayscale = True):
                    mao[1] = mao[1] + 'c'
                elif g.locateOnScreen('PokerStarsImages/espadas3.png', confidence = 0.8, region = (974,670,30,30), grayscale = True):
                    mao[1] = mao[1] + 'e'
                elif g.locateOnScreen('PokerStarsImages/paus3.png', confidence = 0.8, region = (974,670,30,30), grayscale = True):
                    mao[1] = mao[1] + 'p'
                print('mao', mao)
                preFlop()
                
            else:
                # aqui ele detectará as cartas na mesa, vendo seu valor e naipe
                for i in range(5):
                    for j in range(13):
                        arquivo = 'PokerStarsImages/cartas/' + str(j + 2) + '.jpg'
                        if g.locateOnScreen(arquivo, confidence = 0.8, region = (773+i*80,400,32,37), grayscale = True):
                            mesa[i] = str(j+2)
                            break
                    
                    # se alguma carta não for encontrada, o loop se encerrará, pois a próxima carta também não foi jogada ainda
                    if mesa[i] == '00':
                        break

                    if g.locateOnScreen('PokerStarsImages/ouros.png', confidence = 0.75, region = (775+i*80,432,32,36)):
                        mesa[i] = mesa[i] + 'o'
                    elif g.locateOnScreen('PokerStarsImages/espadas.png', confidence = 0.9, region = (775+i*80,432,32,36)):
                        mesa[i] = mesa[i] + 'e'    
                    elif g.locateOnScreen('PokerStarsImages/copas.png', confidence = 0.9, region = (775+i*80,432,32,36)):
                        mesa[i] = mesa[i] + 'c'
                    elif g.locateOnScreen('PokerStarsImages/paus.png', confidence = 0.8, region = (775+i*80,432,32,36)):
                        mesa[i] = mesa[i] + 'p'
                print('mesa', mesa)
                
                # com base no número de cartas jogadas, será detectado em qual turno estamos
                if mesa[4] != "00":
                    turno = 3
                elif mesa[3] != "00":
                    turno = 2
                elif mesa[2] != "00":
                    turno = 1
                else:
                    turno = 0
                # após definidas várias variáveis, o código irá calcular as probabilidades
                Calcular()
            
Loading()  

Funcionando!!!
pote : 300
pagar : 0
mao ['3o', '8e']
rating : 0
pote : 283
pagar : 0
mesa ['12o', '11o', '8p', '00', '00']
minhas probs:
straight flush : 0.0 %
quadra : 0.09 %
full house : 2.5 %
flush : 4.16 %
sequencia : 1.39 %
trinca : 5.83 %
pares : 37.47 %
par : 48.57 %
nada : 0.0 %
adversarios probs:
straight flush : 0.08 %
quadra : 0.11 %
full house : 2.41 %
flush : 5.54 %
sequencia : 17.05 %
trinca : 6.51 %
pares : 33.11 %
par : 68.75 %
nada : 38.07 %
faltam : 2
[0, 49, 37, 6, 1, 4, 2, 0, 0] [21, 44, 18, 3, 9, 3, 1, 0, 0]
winrate:  45.940000000000005
drawrate:  28.63
pote : 567
pagar : 0
mesa ['12o', '11o', '8p', '6p', '00']
minhas probs:
straight flush : 0.0 %
quadra : 0.0 %
full house : 0.0 %
flush : 0.0 %
sequencia : 0.0 %
trinca : 4.35 %
pares : 26.09 %
par : 69.57 %
nada : 0.0 %
adversarios probs:
straight flush : 0.04 %
quadra : 0.04 %
full house : 1.06 %
flush : 3.64 %
sequencia : 10.57 %
trinca : 4.98 %
pares : 26.89 %
par : 72.5 %
nada : 47.52 %
faltam : 2
[0, 70, 26, 4

In [4]:
g.position()

Point(x=637, y=191)

In [11]:
import pyautogui as g
g.moveTo(100,100)

In [12]:
variavel = 10
if variavel == 10:
    print(variavel)
else:
    print('mada')

10


In [14]:
g.position()

Point(x=1038, y=21)

In [23]:
g.moveTo(1038,21)
g.click()
g.write('gmail.com')